In [ ]:
'''
  this part is responsible for the mounting of data from the drive into the
  notebook and retrieving the testing data and models
'''
from google.colab import drive
drive.mount('/content/drive/')

!cp './drive/My Drive/Graduation project/PistolDetection.h5' /content
!cp './drive/My Drive/Graduation project/ViolenceDetection.h5' /content

!cp './drive/My Drive/Graduation project/Demo/mma.mp4' /content
!cp './drive/My Drive/Graduation project/Demo/running.mpg' /content
!cp './drive/My Drive/Graduation project/Demo/pistol.mp4' /content

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
'''
  This part contains the used libraries and essintial parameters
  such as frames and dimensions
'''
!pip install scikit-video
#Used Libraries

import numpy as np
import cv2
import skvideo.io
import h5py
from google.colab.patches import cv2_imshow
import tensorflow
from tensorflow.keras.models import load_model


#Parameters
NUMBER_OF_FRAMES = 20
IMAGE_SIZE = 160
IMAGE_CHANNELS = 3

In [ ]:
ViolenceDetection = load_model('ViolenceDetection.h5')

In [ ]:
PistolDetection = load_model('PistolDetection.h5')

In [ ]:
'''
  This part is responsible for the cropping of the black frames from the videos
'''
def autocrop(image, threshold=0):
    if len(image.shape) == 3:
        flatImage = np.max(image, 2)
    else:
        flatImage = image
    assert len(flatImage.shape) == 2

    rows = np.where(np.max(flatImage, 0) > threshold)[0]
    if rows.size:
        cols = np.where(np.max(flatImage, 1) > threshold)[0]
        image = image[cols[0]: cols[-1] + 1, rows[0]: rows[-1] + 1]
    else:
        image = image[:1, :1]

    return image

In [ ]:
'''
  This part is responsible for resizing and scaling each frame of a video
  - changing the image from BGR to RGB
  - the scaling is from 0 to 1 by dividing each pixel value by 255
  - resizing the images to the expected size in the parameters 
  - applying the autocrop method
'''
def scale_and_resize(image):
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  image = autocrop(image)
  image = cv2.resize(image,(IMAGE_SIZE,IMAGE_SIZE))
  image = image/255.0
  return image.astype('float16')

In [ ]:
'''
  Input parameter: Video name
  Output: tensor of (SAMPLES,10,160,160,3) contains videos divided into frames 
  difference & array of all 20 frames for pistol detection
'''
def extract_frames(video_name):
  output_videos = []

  videodata = skvideo.io.vread(video_name)
  steps = videodata.shape[0] // NUMBER_OF_FRAMES
  new_images = []
  scaled_images = []
  new_diff = np.zeros((int(NUMBER_OF_FRAMES/2),IMAGE_SIZE,IMAGE_SIZE,IMAGE_CHANNELS))
  count = 0

  for j in range(NUMBER_OF_FRAMES):
      new_images.append(videodata[j*steps])
      scaled_images.append(scale_and_resize(videodata[j*steps]))

  for j in range(0,NUMBER_OF_FRAMES,2):
    new_diff[count] = scale_and_resize(cv2.absdiff(new_images[j],new_images[j+1]))
    count += 1 
  
  output_videos.append(new_diff)
  output_videos = np.array(output_videos)
  scaled_images = np.array(scaled_images)
  return output_videos , scaled_images

In [ ]:
'''
  Input: Video name
  Output: The result of the prediction
'''
def Violence_pistol_detection(videoname):
  vid, img = extract_frames(videoname)

  violence_res = ViolenceDetection(vid)
  if violence_res > 0.6:
    print("Violence Detected")
  else:
    print("No Violence Detected")

  pistol_res = PistolDetection(img)
  
  count = 0
  for i in range(len(pistol_res)):
    if pistol_res[i] > 0.5:
      count += 1

  if count > 5:
    print("Pistol Detected")
  else:
    print("No Pistol Detected")

In [ ]:
Violence_pistol_detection('mma.mp4')

Violence Detected
No Pistol Detected


In [ ]:
Violence_pistol_detection('pistol.mp4')

No Violence Detected
Pistol Detected


In [ ]:
Violence_pistol_detection('running.mpg')

No Violence Detected
No Pistol Detected
